Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 84.0 MB/s eta 0:00:00


A — Örnek 1:
∣
𝜓
⟩
=
1
2
(
∣
0
⟩
+
∣
1
⟩
)
∣ψ⟩=
2
	​

1
	​

(∣0⟩+∣1⟩) için klasik kopya vs CNOT sonucu
Cirq (matris + statevector ile)

In [2]:
import numpy as np
import cirq

# tek-qubit |0> ve |1>
ket0 = np.array([1,0], dtype=complex)
ket1 = np.array([0,1], dtype=complex)

# psi = (|0> + |1>) / sqrt(2)
psi = (ket0 + ket1) / np.sqrt(2)

# klasik (ideal) kopya: psi ⊗ psi
classical_clone = np.kron(psi, psi)

# CNOT matrisi (Cirq)
cnot_mat = cirq.unitary(cirq.CNOT)
# başlangıç |psi>|0>
init = np.kron(psi, ket0)
cnot_result = cnot_mat @ init

print("Cirq — psi ⊗ psi (klasik kopya):")
print(np.round(classical_clone, 6))
print("\nCirq — CNOT (|psi>|0>) sonucu:")
print(np.round(cnot_result, 6))


Cirq — psi ⊗ psi (klasik kopya):
[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]

Cirq — CNOT (|psi>|0>) sonucu:
[0.707107+0.j 0.      +0.j 0.      +0.j 0.707107+0.j]


Qiskit (Statevector ile)

In [3]:
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

# psi
psi = (1/np.sqrt(2)) * np.array([1,1], dtype=complex)

# klasik kopya:
classical_clone = np.kron(psi, psi)

# Qiskit: initialize psi on qubit 0, qubit1 is |0>, apply CX
qc = QuantumCircuit(2)
qc.initialize(psi, 0)
qc.cx(0,1)

state_after = Statevector.from_instruction(qc).data

print("Qiskit — psi⊗psi (klasik):", np.round(classical_clone,6))
print("Qiskit — CNOT(|psi>|0>):", np.round(state_after,6))


Qiskit — psi⊗psi (klasik): [0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
Qiskit — CNOT(|psi>|0>): [0.707107+0.j 0.      +0.j 0.      +0.j 0.707107+0.j]


QuTiP (tensor ile)

In [4]:
import numpy as np
from qutip import basis, tensor
from qutip.qip.operations import cnot

# psi (QuTiP Qobj)
psi_q = (basis(2,0) + basis(2,1)).unit()
classical_clone_q = tensor(psi_q, psi_q)

# başlangıç psi ⊗ |0>
init = tensor(psi_q, basis(2,0))
final = cnot() * init

print("QuTiP — klasik clone (psi⊗psi):\n", classical_clone_q)
print("QuTiP — CNOT(|psi>|0>):\n", final)


ImportError: Importing 'qutip.qip' requires the 'qutip_qip' package. Install it with `pip install qutip-qip` (for more details, go to https://qutip-qip.readthedocs.io/).

B — Örnek 2: İç çarpım
⟨
0
∣
𝜙
⟩
⟨0∣ϕ⟩ ve clonability
Tekrar (her kütüphane için numpy yeterli; burada ortak küçük kod)

In [5]:
import numpy as np

ket0 = np.array([1,0], complex)
phi = (1/np.sqrt(2)) * np.array([1,1], complex)  # (|0>+|1>)/sqrt(2)

overlap = np.vdot(ket0, phi)   # <0|phi>
print("<0|phi> =", overlap, " (abs =", abs(overlap), ")")


<0|phi> = (0.7071067811865475+0j)  (abs = 0.7071067811865475 )


C — Örnek 3: Baz durumları (|0>, |1>) CNOT ile “kopyalanabilir” — no-cloning ile çelişmez
Cirq + Qiskit + QuTiP (tek örnek gösterim; burada numpy gösterimi)

In [6]:
import numpy as np
import cirq

ket0 = np.array([1,0], complex)
ket1 = np.array([0,1], complex)

cnot = cirq.unitary(cirq.CNOT)

# Uygula: |0>|0> -> ?
res00 = cnot @ np.kron(ket0, ket0)
res10 = cnot @ np.kron(ket1, ket0)

print("CNOT |0>|0> =", res00)  # beklenen |00>
print("CNOT |1>|0> =", res10)  # beklenen |11>


CNOT |0>|0> = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
CNOT |1>|0> = [0.+0.j 0.+0.j 0.+0.j 1.+0.j]


D — Örnek 4: Genel iki durum için overlap ilişkisi sayısal örnek

(İspatın sayısal sınaması:
⟨
𝜓
1
∣
𝜓
2
⟩
⟨ψ
1
	​

∣ψ
2
	​

⟩ ve
⟨
𝜓
1
⊗
𝜓
1
∣
𝜓
2
⊗
𝜓
2
⟩
⟨ψ
1
	​

⊗ψ
1
	​

∣ψ
2
	​

⊗ψ
2
	​

⟩)

In [7]:
import numpy as np

# iki keyfi normalized durum
psi1 = np.array([0.8, 0.6j]); psi1 = psi1/np.linalg.norm(psi1)
psi2 = np.array([0.3+0.4j, 0.85]); psi2 = psi2/np.linalg.norm(psi2)

ov = np.vdot(psi1, psi2)   # <psi1|psi2>
ov_tensor = np.vdot(np.kron(psi1, psi1), np.kron(psi2, psi2))  # <psi1⊗psi1 | psi2⊗psi2>

print("<psi1|psi2> =", ov)
print("<psi1⊗psi1 | psi2⊗psi2> =", ov_tensor)
print("ov_tensor should equal ov**2 :", ov**2)
print("difference:", ov_tensor - ov**2)


<psi1|psi2> = (0.24336966075042904-0.19266764809408965j)
<psi1⊗psi1 | psi2⊗psi2> = (0.02210796915167096-0.09377892030848323j)
ov_tensor should equal ov**2 : (0.02210796915167095-0.09377892030848328j)
difference: (6.938893903907228e-18+5.551115123125783e-17j)
